In [27]:
import numpy as np
import pandas as pd

In [28]:
#temporada: verano, 1 es verano, 0 es invierno.
inv_verano = pd.read_excel('../02_data/verano.xlsx', sheet_name='veranos', skiprows=1)
inv_verano = pd.melt(inv_verano, id_vars=['zona'], value_vars=inv_verano.columns[1:] , var_name='mes_no', value_name='temporada')

In [29]:
import datetime
import dateutil.relativedelta as relativedelta

mes_inicio = 9
año_inicio = 2023

date_inicio = datetime.datetime(year=año_inicio, month=mes_inicio, day=10)

periodos = []
horizonte_planificacion = 12
for mes in range(0, horizonte_planificacion + 1):
    aux_per = date_inicio + relativedelta.relativedelta(months=mes)
    periodos.append(str(aux_per.year) + '-' + str(aux_per.month).zfill(2))

periodos = pd.DataFrame(periodos, columns=['mes_real'])
periodos['año'] = periodos['mes_real'].str[:4]
periodos['mes_no'] = periodos['mes_real'].str[5:7].astype(int)
periodos['mes_planificado'] = periodos.index
periodos = pd.merge(periodos, inv_verano, how='left', on='mes_no')
periodos.head()

,mes_real,año,mes_no,mes_planificado,zona,temporada
0,2023-09,2023,9,0,chillan,0
1,2023-09,2023,9,0,constitucion,0
2,2023-09,2023,9,0,arauco,0
3,2023-09,2023,9,0,valdivia,0
4,2023-10,2023,10,1,chillan,1


In [30]:
#Carga de plantas

base_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='plantas')
display(base_plantas.head())

#Lectura de parametros planta
parametros_planta = pd.read_excel('../02_data/plantas.xlsx', sheet_name='parametros_planta')
display(parametros_planta.head())

#Lectura consumo plantas
consumo_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='consumo_plantas')
consumo_plantas = pd.melt(consumo_plantas, consumo_plantas.columns[:2], consumo_plantas.columns[2:], var_name='mes_planificado', value_name='volumen_consumo')
display(consumo_plantas.head())


,nombre_planta,producto
0,planta1,EUNI SC
1,planta1,EUNI CC
2,planta1,EUGL SC
3,planta1,EUGL CC


,nombre_planta,producto,capacidad,stock_seguridad_planta,stock_inicial
0,planta1,EUNI SC,40,20,20
1,planta1,EUNI CC,40,20,20
2,planta1,EUGL SC,40,20,20
3,planta1,EUGL CC,40,20,20


,nombre_planta,producto,mes_planificado,volumen_consumo
0,planta1,EUNI SC,1,100
1,planta1,EUNI CC,1,100
2,planta1,EUGL SC,1,100
3,planta1,EUGL CC,1,100
4,planta1,EUNI SC,2,100


In [31]:
#Carga Ingresos: Compras y produccion
base_ingresos = pd.read_excel('../02_data/ingresos.xlsx', sheet_name='ingresos')
base_ingresos.head()

,id_ingreso,nombre_ingreso,nombre_planta,producto,zona,costo existencia,costo transporte,tipo_ingreso,almacen_verano,almacen_invierno,tipo_almacen,cosecha_compra
0,1,produccion EUNI SC constitucion,planta1,EUNI SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
1,2,produccion EUNI arauco sc,planta1,EUNI SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha
2,3,produccion EUNI valdivia sc,planta1,EUNI SC,valdivia,10,20,directo_planta,N_A,N_A,bosque,cosecha
3,4,produccion EUGL constitucion sc,planta1,EUGL SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
4,5,produccion EUGL arauco sc,planta1,EUGL SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha


In [32]:
#Carga Ingreso: Volumen mensual
base_ingreso_vol = pd.read_excel('../02_data/ingresos_volumen.xlsx', sheet_name='ingresovolumen')
base_ingreso_vol.head()

,id_ingreso,mes,volumen
0,1,1,20
1,1,2,30
2,1,3,40
3,2,1,20
4,2,2,30


In [33]:
rango_mes_tipo = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='rango_mes_tipo')
rangos_demanda = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='parametro_antiguedad')

rangos_demanda_ext = pd.DataFrame(columns=['id_tipo_rango', 'rango', 'antiguedad'])
for ix in rangos_demanda.index:
    for ant in range(rangos_demanda.at[ix, 'inicio_rango'], rangos_demanda.at[ix, 'fin_rango'] + 1):
        aux = [[rangos_demanda.at[ix, 'id_tipo_rango'], rangos_demanda.at[ix, 'rango'], ant ]]
        aux = pd.DataFrame(aux, columns=['id_tipo_rango', 'rango', 'antiguedad'])
        rangos_demanda_ext = pd.concat([rangos_demanda_ext, aux], ignore_index=True)

rangos_demanda_ext['antiguedad'] = rangos_demanda_ext['antiguedad'].astype(np.int64).astype(str)
rangos_demanda_ext.head()

,id_tipo_rango,rango,antiguedad
0,1,0-3,0
1,1,0-3,1
2,1,0-3,2
3,1,0-3,3
4,1,4-6,4


In [34]:
## Matriz de demandas.

#Consumo_planta_tipo + parametros_planta_tipo
#Calculo demanda
df = pd.merge(consumo_plantas, parametros_planta, on=['nombre_planta', 'producto'])
df.loc[df['mes_planificado'] != 1, 'stock_inicial'] = 0
df['demanda'] = df['volumen_consumo'] + df['stock_seguridad_planta'] - df['stock_inicial']

#Agregar los tipos de rango por mes
df = pd.merge(df, rango_mes_tipo, how='left', on=['nombre_planta', 'producto', 'mes_planificado'])
df = pd.merge(df, rangos_demanda, how='left', on='id_tipo_rango')
df['demanda_rango'] = df['demanda'] * df['porcentaje']
df = df[['nombre_planta', 'producto', 'mes_planificado', 'id_tipo_rango', 'rango', 'inicio_rango', 'fin_rango', 'demanda_rango']].dropna()
df['id_tipo_rango'] = df['id_tipo_rango'].astype(int)
df['inicio_rango'] = df['inicio_rango'].astype(int)
df['fin_rango'] = df['fin_rango'].astype(int)

done_demanda = df.copy()
done_demanda

,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango
36,planta1,EUGL CC,1,1,0-3,0,3,70.0
37,planta1,EUGL CC,1,1,4-6,4,6,20.0
38,planta1,EUGL CC,1,1,7-9,7,9,10.0
39,planta1,EUGL CC,2,2,0-3,0,3,84.0
40,planta1,EUGL CC,2,2,4-6,4,6,12.0
41,planta1,EUGL CC,2,2,7-13,7,13,24.0
42,planta1,EUGL CC,3,3,0-3,0,3,72.0
43,planta1,EUGL CC,3,3,4-6,4,6,24.0
44,planta1,EUGL CC,3,3,7-9,7,9,12.0
45,planta1,EUGL CC,3,3,10-13,10,13,12.0


In [35]:
### Carga de Almacenes ###
base_almacenes = pd.read_excel('../02_data/Almacenes.xlsx', sheet_name='almacenes')
no_almacenes = len(base_almacenes)

alm_mes_temp = base_almacenes[['id_almacen', 'zona']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp = pd.merge(alm_mes_temp, periodos, how='left', on='zona')
alm_mes_temp = alm_mes_temp[['id_almacen', 'mes_real', 'mes_planificado', 'temporada']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp

,id_almacen,mes_real,mes_planificado,temporada
0,alm1,2023-09,0,0
1,alm1,2023-10,1,1
2,alm1,2023-11,2,1
3,alm1,2023-12,3,1
4,alm1,2024-01,4,1
...,...,...,...,...
307,alm24,2024-05,8,0
308,alm24,2024-06,9,0
309,alm24,2024-07,10,0
310,alm24,2024-08,11,0


In [36]:
#Stock almacen EN MES 0
base_stock_almacenes = pd.read_excel('../02_data/stock_almacen.xlsx', sheet_name='cierrestock')
done_alm = pd.merge(base_almacenes, base_stock_almacenes, how='left', on='id_almacen')
done_alm['ant_meses'] = done_alm['ant_meses'].astype(np.int64).astype(str)
done_alm = done_alm.rename(columns={'ant_meses':'antiguedad'})

done_alm = pd.merge(done_alm, rangos_demanda_ext, on='antiguedad', how='left')
done_alm

,id_almacen,nombre_almacen,nombre_planta,producto,zona,id_almacen_acanche,carpeta,cosecha_compra,tipo_almacen,costo_existencia,capacidad,antiguedad,cierre_stock,id_tipo_rango,rango
0,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,1,1,1,0-3
1,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,1,1,2,0-3
2,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,1,1,3,0-3
3,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,2,1,1,0-3
4,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100,2,1,2,0-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,11,1,3,10-13
836,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,12,1,2,7-13
837,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,12,1,3,10-13
838,alm24,Bosque cosecha EUNI cc RIPIO,planta1,EUNI CC,chillan,alm12,ripio,cosecha,bosque,10,100,13,1,2,7-13


In [37]:
primer_periodo = str(date_inicio.year) + '-' + str(date_inicio.month).zfill(2)
base_stock_almacenes = base_stock_almacenes.rename(columns={'cierre_stock': primer_periodo})
base_stock_almacenes

,id_almacen,ant_meses,2023-09
0,alm1,1,1
1,alm1,2,1
2,alm1,3,1
3,alm1,4,1
4,alm1,5,1
...,...,...,...
307,alm24,9,1
308,alm24,10,1
309,alm24,11,1
310,alm24,12,1


In [38]:
#Armar matriz de almacen-antiguedad-mes
almacenes = done_alm[['id_almacen']].drop_duplicates().reset_index(drop=True)
antiguedades = pd.DataFrame(range(1,13+1), columns=['antiguedad'])
periodoz = periodos[['mes_real']].drop_duplicates().reset_index(drop=True)

almacenes = pd.merge(almacenes, antiguedades, how='cross')
almacenes = pd.merge(almacenes, periodoz, how='cross')
almacenes['stock'] = 0
almacenes = almacenes.pivot_table(index=['id_almacen', 'antiguedad'], columns=['mes_real'], values='stock').rename_axis(columns=None).reset_index()
almacenes

,id_almacen,antiguedad,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09
0,alm1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,alm1,2,0,0,0,0,0,0,0,0,0,0,0,0,0
2,alm1,3,0,0,0,0,0,0,0,0,0,0,0,0,0
3,alm1,4,0,0,0,0,0,0,0,0,0,0,0,0,0
4,alm1,5,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,alm9,9,0,0,0,0,0,0,0,0,0,0,0,0,0
308,alm9,10,0,0,0,0,0,0,0,0,0,0,0,0,0
309,alm9,11,0,0,0,0,0,0,0,0,0,0,0,0,0
310,alm9,12,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
df

,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango
36,planta1,EUGL CC,1,1,0-3,0,3,70.0
37,planta1,EUGL CC,1,1,4-6,4,6,20.0
38,planta1,EUGL CC,1,1,7-9,7,9,10.0
39,planta1,EUGL CC,2,2,0-3,0,3,84.0
40,planta1,EUGL CC,2,2,4-6,4,6,12.0
41,planta1,EUGL CC,2,2,7-13,7,13,24.0
42,planta1,EUGL CC,3,3,0-3,0,3,72.0
43,planta1,EUGL CC,3,3,4-6,4,6,24.0
44,planta1,EUGL CC,3,3,7-9,7,9,12.0
45,planta1,EUGL CC,3,3,10-13,10,13,12.0


In [46]:
#Modifica demanda del  primer periodo de acuerdo a disponibilidad.

for planta in df['nombre_planta'].unique():
    for producto in df[df['nombre_planta'] == planta]['producto'].unique():
        for mes in [1]:
            #Para cada (planta, producto, mes) se filtra la demanda requerida.
            aux = df[(df['nombre_planta'] == planta) & (df['producto'] == producto) & (df['mes_planificado'] == mes)]
            aux = aux.sort_values(by='rango', ascending=False).reset_index(drop=True)
            # display(aux)
            #En una (planta, producto, mes) solo puede haber un "id_tipo_rango".
            tipo_rango_mes = aux['id_tipo_rango'].unique()[0]

            #Se verifica que exista en los almacenes la cantidad de producto disponible por rango-antiguedad y con disponibilidad en la temporada.
            new_alm = done_alm.copy()
            new_alm = new_alm[(new_alm['nombre_planta'] == planta) 
                                & (new_alm['producto'] == producto)
                                & (new_alm['id_tipo_rango'] == tipo_rango_mes)]

            #Temporada del almacen
            temp_alm = alm_mes_temp.copy()
            temp_alm = temp_alm[(temp_alm['mes_planificado'] == mes)]

            new_alm = pd.merge(new_alm, temp_alm, how='left', on='id_almacen')
            inv_drops = new_alm.loc[(new_alm['temporada'] == 0) & (new_alm['carpeta'] == 'tierra')].index
            new_alm = new_alm.drop(inv_drops, axis=0)
            new_alm = new_alm.groupby(['nombre_planta', 'producto', 'rango'], as_index=False).agg(volumen=('cierre_stock', 'sum'))
            new_alm = new_alm.sort_values(by='rango', ascending=False).reset_index(drop=True)

            aux = pd.merge(aux, new_alm, how='left', on=['nombre_planta', 'producto', 'rango'])
            display(aux)

            for ix in aux.index[:-1]:
                if aux.at[ix, 'volumen'] < aux.at[ix, 'demanda_rango']:
                    aux.at[ix + 1, 'demanda_rango'] +=  aux.at[ix, 'demanda_rango'] - aux.at[ix, 'volumen']
                    aux.at[ix, 'demanda_rango'] = aux.at[ix, 'volumen']
            
display(aux)


,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango,volumen
0,planta1,EUGL CC,1,1,7-9,7,9,10.0,18
1,planta1,EUGL CC,1,1,4-6,4,6,20.0,18
2,planta1,EUGL CC,1,1,0-3,0,3,70.0,18


,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango,volumen
0,planta1,EUGL CC,1,1,7-9,7,9,10.0,18
1,planta1,EUGL CC,1,1,4-6,4,6,18.0,18
2,planta1,EUGL CC,1,1,0-3,0,3,72.0,18
